# Hierarchical Binary Classifiers
The premise of this model is to instead of identifying all 4 classes at once, identify whether a stroke is long axis vs short axis. Then, based on that decision determine whether the short axis stroke is breastroke or butterfly, and whether the long axis stroke is freestyle or backstroke. The reason for this specific hierarchical tree is because the model was facing significant issues differentiating between freestyle and backstroke, which was somewhat alleviated by introducing a DNN rather than a CNN


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, optimizers
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import Callback
from tensorflow import Tensor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.initializers import he_normal

from IPython.display import clear_output

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold


import datetime
import os
import matplotlib.pyplot as plt

#import sklearn
#from sklearn.metrics import confusion_matrix
import math

In [2]:
num_classes = 2

x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')
x_test = np.load('x2_test.npy')
y_test = np.load('y2_test.npy')
xt = x_test
yt = y_test
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

yt = keras.utils.to_categorical(yt, num_classes)
tl = np.argmax(yt, axis=1)


(2400, 32, 12, 3)
(2400, 2)
(97, 32, 12, 3)
(97, 2)


In [3]:
longAxisBC0 = tf.keras.models.load_model('testModel74')
longAxisBC1 = tf.keras.models.load_model('testModel75')
longAxisBC2 = tf.keras.models.load_model('testModel76')
longAxisBC3 = tf.keras.models.load_model('testModel77')
#longAxisBC4 = tf.keras.models.load_model('testModel78')

shortAxisBC0 = tf.keras.models.load_model('testModel81')
shortAxisBC1 = tf.keras.models.load_model('testModel82')
shortAxisBC2 = tf.keras.models.load_model('testModel83')
shortAxisBC3 = tf.keras.models.load_model('testModel84')
shortAxisBC4 = tf.keras.models.load_model('testModel85')

axisBC0 = tf.keras.models.load_model('testModel90')
axisBC1 = tf.keras.models.load_model('testModel91')
axisBC2 = tf.keras.models.load_model('testModel92')
axisBC3 = tf.keras.models.load_model('testModel93')
axisBC4 = tf.keras.models.load_model('testModel94')

"test.evaluate(x_test, y_test, verbose=1)\npredicted_labels=np.argmax(test.predict(xt), axis=1)\ntl = np.argmax(yt, axis=1)\n#tl = yt\nconfusion_mat = confusion_matrix(tl, predicted_labels)\nprint(confusion_mat)\nprint(f'Model Accuracy based on testing data: {(np.trace(confusion_mat))/np.sum(confusion_mat)}')\n"

In [21]:
shortAxisBC1.evaluate(x_test, y_test, verbose=1)
predicted_labels=np.argmax(shortAxisBC0.predict(xt), axis=1)
tl = np.argmax(yt, axis=1)
#tl = yt
confusion_mat = confusion_matrix(tl, predicted_labels)
print(confusion_mat)
print(f'Model Accuracy based on testing data: {(np.trace(confusion_mat))/np.sum(confusion_mat)}')


4/4 [==============================] - 0s 29ms/step - loss: 0.6413 - accuracy: 0.6804
[[41 11]
 [27 18]]
Model Accuracy based on testing data: 0.6082474226804123


In [28]:
#Hard Voting Ensemble
#Step 1: test axis
y_check = []
count = 0
for data in x_test:
    x1 = np.argmax(shortAxisBC0.predict(np.reshape(data, (1, 32, 12, 3))), axis=1)
    x2 = np.argmax(shortAxisBC1.predict(np.reshape(data, (1, 32, 12, 3))), axis=1)
    x3 = np.argmax(shortAxisBC2.predict(np.reshape(data, (1, 32, 12, 3))), axis=1)
    x4 = np.argmax(shortAxisBC3.predict(np.reshape(data, (1, 32, 12, 3))), axis=1)
    x5 = np.argmax(shortAxisBC4.predict(np.reshape(data, (1, 32, 12, 3))), axis=1)
    array = [x1, x2, x3, x4, x5]
    c=0
    count += 1
    for i in array:
        if i == [0]:
            c+=1
        else:
            pass
    if c >=3:
        y_check.append(0)
    else:
        y_check.append(1)
    if count%100 == 0:
        print("100 count")

In [6]:
conf = [[0.3, 0.7], [0.2, 0.8], [0.5, 0.5], [0.7, 0.3], [0.1, 0.9]]

num_models = len(conf)  # Number of models

# Initialize a dictionary to store accumulated class probabilities
conf_array = np.array(conf)

# Calculate the average of values at index 0 and index 1
conf_array = [np.mean(conf_array[:, 0]), np.mean(conf_array[:, 1])]  # Average of index 0 values
print(np.argmax(conf_array))

1


In [26]:
#Step 1: test axis
y_check = []
count = 0
for data in x_test:
    #x1 = shortAxisBC0.predict(np.reshape(data, (1, 32, 12, 3)))
    x2 = shortAxisBC1.predict(np.reshape(data, (1, 32, 12, 3)))
    x3 = shortAxisBC2.predict(np.reshape(data, (1, 32, 12, 3)))
    x4 = shortAxisBC3.predict(np.reshape(data, (1, 32, 12, 3)))
    x5 = shortAxisBC4.predict(np.reshape(data, (1, 32, 12, 3)))
    conf = [x2, x3, x4, x5]
    count += 1

    num_models = len(conf)  # Number of models

    # Initialize a dictionary to store accumulated class probabilities
    conf_array = np.array(conf).reshape(-1, 2)

    # Calculate the average of values at index 0 and index 1
    conf_array = [np.mean(conf_array[:, 0]), np.mean(conf_array[:, 1])]  # Average of index 0 values
    y_check.append(np.argmax(conf_array))
    if count%10 == 0:
        print("10 count")

10 count
10 count
10 count
10 count
10 count
10 count
10 count
10 count
10 count


In [29]:
predicted_labels = np.array(y_check)
tl = np.argmax(yt, axis=1)
confusion_mat = confusion_matrix(tl, predicted_labels)
print(confusion_mat)
print(f'Model Accuracy based on testing data: {(np.trace(confusion_mat))/np.sum(confusion_mat)}')

[[45  7]
 [11 34]]
Model Accuracy based on testing data: 0.8144329896907216


### Long Axis Series (Free vs Back Classifier)
All ensemble types showed no signs of improvement

### Short Axis Series (Fly vs Breast Classifier)
Original
approx 72
##### Hard Voting Ensemble
eh

##### Soft Voting Ensemble
[[45  7]
 [11 34]]
Model Accuracy based on testing data: 0.8144329896907216

### Axis BC Series (FB vs FB)
Original
[[77 18]
 [27 70]]
Model Accuracy based on testing data: 0.765625
##### Hard Voting Ensemble
[[84 11]
 [30 67]]
Model Accuracy based on testing data: 0.7864583333333334
##### Soft Voting Ensemble
[[85 10]
 [32 65]]
Model Accuracy based on testing data: 0.78125